In [1]:
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import NoSuchElementException  #Este importé para manejar más errores.
from time import sleep
import os
import signal
import sys

In [2]:
df_peliculas = pd.read_csv("peliculas_api.csv")

df_peliculas.head(10)

,Unnamed: 0,Tipo,Nombre,Año de Estreno,Mes de Estreno,ID de la Película,Genero
0,0,Movie,Pál Adrienn,2011,6,tt0146592,Accion/Drama
1,1,Movie,Oda az igazság,2010,2,tt0154039,Accion/Drama
2,2,Movie,A zöld sárkány gyermekei,2011,4,tt0162942,Accion/Drama
3,3,Movie,The Final Journey,2010,9,tt0230212,Accion/Drama
4,4,Movie,In My Sleep,2010,4,tt0326965,Accion/Drama
5,5,Movie,This Wretched Life,2010,7,tt0331312,Accion/Comedia/Drama
6,6,Movie,Blind Sided,Desconocido,Desconocido,tt0337882,Accion/Comedia/Drama
7,7,Movie,Anderson's Cross,2010,5,tt0393049,Accion/Comedia/Drama
8,8,Movie,Den milde smerte,2014,5,tt0396123,Accion/Drama
9,9,Movie,Capture the Flag,Desconocido,Desconocido,tt0402887,Accion/Drama


In [3]:
df_peliculas.columns

Index(['Unnamed: 0', 'Tipo', 'Nombre', 'Año de Estreno', 'Mes de Estreno',
       'ID de la Película', 'Genero'],
      dtype='object')

In [4]:
df_id_nombre = df_peliculas[['ID de la Película', 'Nombre']]
df_id_nombre

,ID de la Película,Nombre
0,tt0146592,Pál Adrienn
1,tt0154039,Oda az igazság
2,tt0162942,A zöld sárkány gyermekei
3,tt0230212,The Final Journey
4,tt0326965,In My Sleep
...,...,...
34970,tt31308988,Married Life
34971,tt31314922,Concept High
34972,tt31316077,Satisfaction
34973,tt31319755,Party Pooper


In [5]:
#PREGUNTAS VIOLAINE

def signal_handler(signal, frame, driver, datos_peliculas):
    print("Proceso interrumpido. Guardando datos...")
    df_resultados_temporales = pd.DataFrame(datos_peliculas)
    df_resultados_temporales.to_csv('datos_temporales.csv', index=False)
    print("Datos guardados en 'datos_temporales.csv'.")
    guardar_datos(datos_peliculas)  #  definir esta función
    driver.close()
    sys.exit(0)
    
def guardar_datos(datos_peliculas):
   # Define el  archivo donde vamos a guardar los datos.
    ruta_archivo = 'fase-2-datos-peliculas-imdb-rotten.csv'
    
    # Convierte el diccionario a DataFrame
    df_nuevos_datos = pd.DataFrame(datos_peliculas)
    
    if os.path.exists(ruta_archivo):
        # Si el archivo ya existe, carga los datos existentes y los junta.
        df_existentes = pd.read_csv(ruta_archivo)
        df_resultante = pd.concat([df_existentes, df_nuevos_datos], ignore_index=True)
    else:
        # Si el archivo no existe, utiliza los nuevos datos directamente
        df_resultante = df_nuevos_datos
    
    # Guardamos el DataFrame resultante en el archivo, sobrescribiendo el archivo existente
    df_resultante.to_csv(ruta_archivo, index=False)
    
    print(f"Datos guardados o actualizados exitosamente en {ruta_archivo}.")

# Aqui empieza nuestra función de extraccion de datos 'IMDB' y 'ROTTEN TOMATOES'
def extraccion_imdb_rotten(df_peliculas):
    driver = webdriver.Chrome()
    datos_peliculas = {
        "Nombre": [],
        "Puntuacion_IMDB": [],
        "Puntuacion_Rotten_Tomatoes": [],
        "Director": [],
        "Guionistas": [],
        "Argumento": [],
        "Duracion": []
    }
    
    signal.signal(signal.SIGINT, lambda sig, frame: signal_handler(sig, frame, driver, datos_peliculas))  #Esto es para poder interrumpir la función.

    for indice, fila in df_peliculas[4964:].iterrows(): # Aqui podemas aplicar  el rango de búsqueda usando 'slicing', para no tener que empezar de cero en caso de error o interrupción.
        imdb_id = fila['ID de la Película']
        nombre_pelicula = fila['Nombre']
        try:
            driver.get(f"https://www.imdb.com/title/{imdb_id}")
            sleep(2)
            driver.maximize_window()
            driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
            sleep(1)
#PORQUE HAY DOS VECES EL MISMO EXCEPT ??
        except NoSuchElementException:
            # Si el botón de aceptación de cookies no está presente, simplemente continúa. Lo cual, en este caso,  pasa a partir de la segunda busqueda.
            print("No se requiere aceptación de cookies.")
        except NoSuchElementException: #Herramienta importada de Selenium.
            print(f"No se encontro la pagina para el ID {imdb_id}")
        except Exception as e:
            print(f"Error durante la búsqueda del ID IMDB {imdb_id}: {str(e)}")
         
#PONER EN XPATH EN TODAS PARTES.         
        try:
            puntuacion_imdb = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/div[1]/div/div[1]/a/span/div/div[2]/div[1]/span[1]').text
        except Exception as e:
            print(f"No se encontró la puntuación de {nombre_pelicula} en IMDb: {str(e)}")
            puntuacion_imdb = "N/A"
 # AQUI PORQUE NO HAY UN APPEND ?      
        try:
            director = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div/ul/li/a').text
        except Exception as e:
            print(f"No se encontró el director de {nombre_pelicula} en IMDb: {str(e)}")
            director = "N/A"
        
        try:
            guionistas = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]/div/ul/li/a').text
        except Exception as e:
            print(f"No se encontró guionista de {nombre_pelicula} en IMDb: {str(e)}")
            guionistas = "N/A"
        
        try: 
            argumento = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[2]').text
            
        except Exception as e:
          print(f"No se encontro argumento de {nombre_pelicula} en imdb {str(e)}")
          argumento = "N/A"       

        try:
            duracion = driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[2]').text

        except Exception as e:
                print(f"No se encontro duracion de {nombre_pelicula} en imdb {str(e)}")
                duracion = "N/A"
                
        # ROTTEN TOMATOES buscamos por nombre porque no acepta IDS a diferencia de IMD
        try:  # modificamos el nombre de las pelis para que se adapten a los parametros de búsqueda de 'Rotten Tomatoes'  Ya que está no funciona con búsqueda por ID'S.
            nombre_pelicula_modificado = nombre_pelicula.replace("'","").replace("ñ","n").replace(":","").replace(",","").replace("/","").replace("!","").replace("?","") # porque no me deja hacerlo directamente en la siguiente línea.
            driver.get(f'https://www.rottentomatoes.com/m/{nombre_pelicula_modificado.replace(" ","_").replace("&","and").lower()}')
            sleep(2)
            try:
                driver.find_element("css selector", "#onetrust-accept-btn-handler").click()
            except NoSuchElementException:
            # Si el botón de aceptación de cookies no está presente, simplemente continúa. Lo cual, en este caso,  pasa a partir de la segunda busqueda.
                print("No se requiere aceptación de cookies.")
        
            puntuacion_rotten = driver.find_element("css selector", "#modules-wrap > div.media-scorecard.no-border > media-scorecard > rt-button > rt-text").text
            
        except Exception as e:
            print(f"No se encontro la película {nombre_pelicula} en Rotten Tomatoes: {str(e)}")
            puntuacion_rotten = "N/A"
            

        datos_peliculas['Nombre'].append(nombre_pelicula)
        datos_peliculas['Puntuacion_IMDB'].append(puntuacion_imdb)
        datos_peliculas['Puntuacion_Rotten_Tomatoes'].append(puntuacion_rotten)
        datos_peliculas['Director'].append(director)
        datos_peliculas['Guionistas'].append(guionistas)
        datos_peliculas['Argumento'].append(argumento)
        datos_peliculas['Duracion'].append(duracion)

    driver.close()
    return datos_peliculas

# Llamamos a  la función con nuestro archivo de pelis que sacamos de la API.
df_peliculas = pd.read_csv("peliculas_api.csv")
resultados = extraccion_imdb_rotten(df_peliculas)

No se encontró la puntuación de I Am Not the Devil en IMDb: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/div[1]/div/div[1]/a/span/div/div[2]/div[1]/span[1]"}
  (Session info: chrome=123.0.6312.105); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5e72f7178863 <unknown>
#1 0x5e72f6e6e8c6 <unknown>
#2 0x5e72f6eb9618 <unknown>
#3 0x5e72f6eb96d1 <unknown>
#4 0x5e72f6efc744 <unknown>
#5 0x5e72f6edb5cd <unknown>
#6 0x5e72f6ef9c19 <unknown>
#7 0x5e72f6edb343 <unknown>
#8 0x5e72f6eac593 <unknown>
#9 0x5e72f6eacf5e <unknown>
#10 0x5e72f713c84b <unknown>
#11 0x5e72f71407a5 <unknown>
#12 0x5e72f712a571 <unknown>
#13 0x5e72f7141332 <unknown>
#14 0x5e72f710f87f <unknown>
#15 0x5e72f7167728 <unknown>
#16 0x5e72f71678fb <unknown>
#17 0x5e72f71779b4 <unknown>
#18 0